# modep tabular AutoML

## Install the client

This python client wraps the modep API.

In [3]:
!pip install git+https://github.com/modep-ai/modep-client.git

Obtaining file:///home/jimmie/git/mlapi/modep-client
  Preparing metadata (setup.py) ... done
  Running setup.py develop for modep-client


In [1]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import os 
import uuid
import time
import logging
import numpy as np
import pandas as pd
from IPython.display import display, HTML

from modep_client.client import Client
from modep_client.datasets import Datasets
from modep_client.automl import Frameworks, FrameworkFlights

logging.getLogger().setLevel(logging.INFO)

In [68]:
client = Client('<YOUR API KEY>', url='http://localhost:5000/v1/', ensure_https=False)

datasets = Datasets(client)
frameworks = Frameworks(client)
flights = FrameworkFlights(client)

In [69]:
dsets = datasets.list()
dsets

,name,ext,mbytes,created,is_public,target,categorical_target
id,,,,,,,
f19bb48c-eaa3-4064-8ad2-866725d62375,OpenML-3945-name=KDDCup09_appetency-fold=00-sp...,csv,2.853640,2021-11-18T21:45:43.713005,False,APPETENCY,True
f0fcce94-5afe-4c72-8e83-e535fe08ed36,OpenML-3945-name=KDDCup09_appetency-fold=00-sp...,csv,25.661045,2021-11-18T21:45:40.426640,False,APPETENCY,True


In [59]:
task_id= 3945
dsets_ = dsets[dsets['name'].str.contains(f'id={task_id}')]
train_dset = dsets_[dsets_['name'].str.contains('split=train')]
test_dset = dsets_[dsets_['name'].str.contains('split=test')]

In [60]:
dsets_

,name,ext,mbytes,created,is_public,target,categorical_target
id,,,,,,,
9213d1d9-d576-4e99-bc33-113007c1a561,OpenML-id=3945-name=KDDCup09_appetency-fold=00...,csv,2.853640,2021-11-18T21:14:24.833895,False,None,None
1657c0fa-d6c7-4406-8364-4cb34b408279,OpenML-id=3945-name=KDDCup09_appetency-fold=00...,csv,25.661045,2021-11-18T21:14:21.526179,False,None,None


In [64]:
train_dset.index[0]

'1657c0fa-d6c7-4406-8364-4cb34b408279'

In [67]:
train_dset.iloc[0]['target']

In [62]:
test_dset

,name,ext,mbytes,created,is_public,target,categorical_target
id,,,,,,,
9213d1d9-d576-4e99-bc33-113007c1a561,OpenML-id=3945-name=KDDCup09_appetency-fold=00...,csv,2.85364,2021-11-18T21:14:24.833895,False,None,None


In [38]:
# ## delete all datasets
# for idx in datasets.list().index:
#     print(datasets.delete(idx))

In [39]:
frameworks.list()

""


In [40]:
flights.list()

""


In [18]:
# kaggle dataset name
dataset = 'mercedes-benz-greener-manufacturing'

# location of the data directory
ddir = f'./kaggle/{dataset}'

# the target column name
target = 'y'

# whether the target is categorical
categorical_target = False

## name of datasets to be created from train and test DataFrames
train_name = dataset + '-train'
test_name = dataset + '-test'

In [19]:
!du -h $ddir/*.csv

92K	/home/jimmie/kaggle/mercedes-benz-greener-manufacturing/sample_submission.csv
3.1M	/home/jimmie/kaggle/mercedes-benz-greener-manufacturing/test.csv
3.1M	/home/jimmie/kaggle/mercedes-benz-greener-manufacturing/train.csv


## Get datasets

In [20]:
datasets.upload?

Signature:
datasets.upload(
    dset: Union[str, pandas.core.frame.DataFrame],
    name: str,
    target: str = None,
    categorical_target: bool = True,
)
Docstring:
Upload a tabular dataset.

:param dset: either a path to a CSV file or DataFrame containing the data
:type dset: str or :class:`pandas.DataFrame`
:param str name: A name to give the dataset (ie. `titanic-train` or `titanic-test`)
:param target: Optionally specify a target column for the dataset
:type target: str or None
:param bool categorical_target: `True` if the specified `target` column is categorical
    (for classification), otherwise set this to `False` for regression.
File:      ~/git/mlapi/modep-client/modep_client/datasets.py
Type:      method


In [22]:
dsets = datasets.list()

if len(dsets) == 0 or train_name not in dsets.name.values or test_name not in dsets.name.values:
    # prepare training and test datasets
    
    df_train = pd.read_csv(ddir + '/train.csv')

    # drop any ID column    
    for col in ['id', 'ID']:
        if col in df_train:
            df_train = df_train.drop([col], 1)
    
    if categorical_target:
        # convert integer classes to strings
        int_to_label = {i: f'class_{i}' for i in df_train[target].unique()}
        df_train[target] = [int_to_label[i] for i in df_train[target]]

    # test set for public leaderboard (has no target column)
    df_test = pd.read_csv(ddir + '/test.csv')

    # drop any ID column
    for col in ['id', 'ID']:
        if col in df_test:
            df_test = df_test.drop([col], 1)
    
    # add a dummy target column for the test set, 
    # random sample so that there's more than one of each class
    if target not in df_test.columns:
        df_test[target] = df_train[target].sample(len(df_test)).values
        
    # put the test columns in the same order as the train columns
    df_test = df_test.loc[:, df_train.columns]
        
    # <TODO> remove
    # df_train = df_train.sample(10000)        
    # df_test = df_test.sample(10000)
    
    train_dset = datasets.upload(df_train, train_name, target=target, categorical_target=categorical_target)
    test_dset = datasets.upload(df_test, test_name, target=target, categorical_target=categorical_target)
    dsets = datasets.list()

In [23]:
# get IDs of datasets
train_id = dsets[dsets.name == train_name].index[0]
test_id = dsets[dsets.name == test_name].index[0]
train_id, test_id

('bf30a8e6-9e9e-4f91-8544-0a5cf57dfd90',
 'f92d4be8-6b04-4406-88fa-c5be25f929cb')

In [24]:
dsets

,name,ext,mbytes,created,is_public,target,categorical_target
id,,,,,,,
f92d4be8-6b04-4406-88fa-c5be25f929cb,mercedes-benz-greener-manufacturing-test,csv,3.052346,2021-11-18T19:40:04.529827,False,y,False
bf30a8e6-9e9e-4f91-8544-0a5cf57dfd90,mercedes-benz-greener-manufacturing-train,csv,3.052213,2021-11-18T19:40:00.982874,False,y,False


In [25]:
df_train.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_test.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,98.86,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,110.07,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,86.93,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,92.53,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,90.13,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [34]:
df_train['X5'].value_counts().sort_index()

aa    112
ab    197
ac    200
ad    185
ae    205
af    188
ag    204
ah     97
c     131
d     214
f       7
g       1
h       1
i     207
j     125
k     177
l     195
m     208
n     212
o      20
p     208
q     220
r     215
s     214
u       1
v     231
w     231
x       2
y       1
Name: X5, dtype: int64

In [35]:
df_test['X5'].value_counts().sort_index()

a       1
aa    105
ab    179
ac    212
ad    213
ae    196
af    217
ag    201
ah     80
b       1
c     121
d     192
f       6
g       8
h       2
i     180
j     137
k     193
l     206
m     197
n     209
o      16
p     227
q     197
r     239
s     205
t       1
v     246
w     218
x       2
y       1
z       1
Name: X5, dtype: int64

In [28]:
df_test.head(10)

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,98.86,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,110.07,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,86.93,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,92.53,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,90.13,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
5,100.11,y,aa,ai,e,d,x,g,s,0,...,1,0,0,0,0,0,0,0,0,0
6,89.10,x,b,ae,d,d,x,d,y,0,...,0,0,0,0,0,1,0,0,0,0
7,76.22,f,s,ae,c,d,h,d,a,0,...,0,0,1,0,0,0,0,0,0,0
8,101.70,ap,l,s,c,d,h,j,n,0,...,0,0,0,0,0,0,0,0,0,0
9,101.25,o,v,as,f,d,g,f,v,0,...,0,0,0,0,0,0,0,0,0,0


In [223]:
np.all(np.isfinite(df_test['y']))

True

In [228]:
np.where(df_test['X0'].isnull())

(array([], dtype=int64),)

## Get all of the available AutoML frameworks

- The `has_predict` column indicates whether predicting on new data is supported. Some frameworks like AutoWEKA and GAMA can only be provided with a test set at the time of training.

In [140]:
frameworks.info()

,description,project,params,has_predict
framework_name,,,,
AutoGluon,AutoGluon-Tabular: Unlike existing AutoML fram...,https://auto.gluon.ai,{'_save_artifacts': ['leaderboard']},True
AutoGluon_bestquality,AutoGluon with 'best_quality' preset provides ...,,"{'_save_artifacts': ['leaderboard'], 'presets'...",True
autosklearn,auto-sklearn frees a machine learning user fro...,https://www.automl.org/auto-sklearn,{'_save_artifacts': ['models']},True
autosklearn2,Improved version of the original auto-sklearn ...,https://www.automl.org/auto-sklearn-2-0-the-ne...,"{'_askl2': True, '_save_artifacts': ['models']}",True
AutoWEKA,Auto-WEKA considers the problem of simultaneou...,https://www.cs.ubc.ca/labs/beta/Projects/autow...,None,False
constantpredictor,Fast dummy classifier mainly used to test the ...,https://scikit-learn.org/stable/modules/genera...,None,True
DecisionTree,A simple decision tree implementation (scikit-...,https://scikit-learn.org/stable/modules/genera...,None,True
flaml,FLAML is a lightweight Python library that fin...,https://github.com/microsoft/FLAML,None,True
GAMA,GAMA tries to find a good machine learning pip...,https://github.com/PGijsbers/gama,None,False


In [178]:
for name, info in frameworks.info().iterrows():
    display(HTML(f'''
    <h3>{name}</h3>
    <p>{info.description}</p>
    <p><a href="{info.project}">{info.project}</a></p>
    <p>params: {info.params}</p>
    '''))

## Train each framework

In [141]:
framework_names = frameworks.info().index.tolist()
framework_names

['AutoGluon',
 'AutoGluon_bestquality',
 'autosklearn',
 'autosklearn2',
 'AutoWEKA',
 'constantpredictor',
 'DecisionTree',
 'flaml',
 'GAMA',
 'H2OAutoML',
 'hyperoptsklearn',
 'mljarsupervised',
 'mljarsupervised_compete',
 'MLNet',
 'RandomForest',
 'TPOT',
 'TunedRandomForest']

In [143]:
frameworks.train?

Signature:
frameworks.train(
    framework_name: str,
    train_ids: Union[str, List[str]],
    test_ids: Union[str, List[str]],
    target: str,
    max_runtime_seconds: int,
)
Docstring:
Train an AutoML framework

:param str framework_name: The name of the framework (ie. AutoGluon, AutoGluon_bestquality,
    autosklearn, autosklearn2, AutoWEKA, constantpredictor, DecisionTree, flaml, GAMA, H2OAutoML,
    hyperoptsklearn, mljarsupervised, mljarsupervised_compete, MLNet, RandomForest, TPOT, TunedRandomForest)
:param train_ids: The id(s) of dataset(s) to train on (ie. `e1bc3d16b-6d67-43cd-af59-8d39d8cb2a02`)
:param test_ids: The id(s) of dataset(s) to test on (ie. `1bc3d16b-6d67-43cd-af59-8d39d8cb2a02`)
:param str target: The name of the target column in the training dataset(s)
:param int max_runtime_seconds: The maximum amount of time in seconds to train per dataset(s)
:return: A :class:`modep_client.tasks.BaseTask` object
File:      ~/git/mlapi/modep_client/modep_client/automl.py
Type

In [245]:
train_job = frameworks.train('autosklearn', train_id, test_id, target, 60*2)
train_job

INFO:modep_client.automl:{'framework_name': 'autosklearn', 'train_ids': ['4378de67-d256-4ed7-8653-82758ee1a343'], 'test_ids': ['8042af36-76c0-4ba5-a7d5-af581d62e916'], 'target': 'y', 'max_runtime_seconds': 120, 'experiment_id': ''}


In [246]:
## status is CREATED
train_job.response

{'id': 'f7f41981-463d-4ba5-b36d-d4b1951def71',
 'framework_name': 'autosklearn',
 'version': None,
 'train_ids': ['4378de67-d256-4ed7-8653-82758ee1a343'],
 'test_ids': ['8042af36-76c0-4ba5-a7d5-af581d62e916'],
 'target': 'y',
 'max_runtime_seconds': 120,
 'created': '2021-11-16T22:40:46.234425',
 'status': 'CREATED',
 'problem_type': None,
 'metric_name': None,
 'metric_value': None,
 'other_metrics': None,
 'duration': None,
 'training_duration': None,
 'predict_duration': None,
 'models_count': None,
 'n_folds': 1,
 'fold_results': None,
 'fold_leaderboard': None,
 'fold_model_txt': None,
 'experiment_id': '',
 'info': ''}

In [249]:
## status is RUNNING
frameworks.get(train_job.response['id'])

{'id': 'f7f41981-463d-4ba5-b36d-d4b1951def71',
 'framework_name': 'autosklearn',
 'version': '0.13.0',
 'train_ids': ['4378de67-d256-4ed7-8653-82758ee1a343'],
 'test_ids': ['8042af36-76c0-4ba5-a7d5-af581d62e916'],
 'target': 'y',
 'max_runtime_seconds': 120,
 'created': '2021-11-16T22:40:46.234425',
 'status': 'FAIL',
 'problem_type': 'regression',
 'metric_name': 'neg_rmse',
 'metric_value': None,
 'other_metrics': {'rmse': None, 'r2': None, 'mae': None},
 'duration': 0.9,
 'training_duration': 0.0,
 'predict_duration': 0.0,
 'models_count': 0,
 'n_folds': 1,
 'fold_results': [{'framework': 'autosklearn',
   'version': '0.13.0',
   'fold': 0,
   'type': 'regression',
   'result': None,
   'metric': 'neg_rmse',
   'duration': 0.9,
   'training_duration': None,
   'predict_duration': None,
   'models_count': None,
   'seed': 318419386,
   'info': "ValueError: Found unknown categories ['none'] in column 0 during transform",
   'rmse': None,
   'r2': None,
   'mae': None}],
 'fold_leaderb

In [248]:
## wait for training job to finish
train_job.result()

{'id': 'f7f41981-463d-4ba5-b36d-d4b1951def71',
 'framework_name': 'autosklearn',
 'version': '0.13.0',
 'train_ids': ['4378de67-d256-4ed7-8653-82758ee1a343'],
 'test_ids': ['8042af36-76c0-4ba5-a7d5-af581d62e916'],
 'target': 'y',
 'max_runtime_seconds': 120,
 'created': '2021-11-16T22:40:46.234425',
 'status': 'FAIL',
 'problem_type': 'regression',
 'metric_name': 'neg_rmse',
 'metric_value': None,
 'other_metrics': {'rmse': None, 'r2': None, 'mae': None},
 'duration': 0.9,
 'training_duration': 0.0,
 'predict_duration': 0.0,
 'models_count': 0,
 'n_folds': 1,
 'fold_results': [{'framework': 'autosklearn',
   'version': '0.13.0',
   'fold': 0,
   'type': 'regression',
   'result': None,
   'metric': 'neg_rmse',
   'duration': 0.9,
   'training_duration': None,
   'predict_duration': None,
   'models_count': None,
   'seed': 318419386,
   'info': "ValueError: Found unknown categories ['none'] in column 0 during transform",
   'rmse': None,
   'r2': None,
   'mae': None}],
 'fold_leaderb

In [236]:
frameworks.get_output(train_job.response['id'])

'/tmp/tmps8th3n0r-modep-output'

In [237]:
frameworks.print_log(train_job.response['id'])

[INFO] [amlb:21:21:25.504] Running benchmark `autosklearn` on `benchmark` framework in `local` mode.
[DEBUG] [amlb:21:21:25.504] Script args: Namespace(benchmark='benchmark', constraint='constraint', exit_on_error=False, extra=[], fold=None, framework='autosklearn', indir=None, keep_scores=True, logging='debug', mode='local', outdir='/tmp/tmpdyw0fb0t', parallel=1, profiling=False, session=None, setup='auto', task=None, userdir='/tmp/tmpdyw0fb0t').
[DEBUG] [amlb.utils.config:21:21:25.504] Loading config file `/bench/automlbenchmark/resources/config.yaml`.
[DEBUG] [amlb.utils.config:21:21:25.557] Loading config file `/tmp/tmpdyw0fb0t/config.yaml`.
[DEBUG] [amlb:21:21:25.559] Config args: { 'exit_on_error': False,
  'output_dir': '/tmp/tmpdyw0fb0t',
  'parallel_jobs': 1,
  'results': {'save': True},
  'run_mode': 'local',
  'script': 'runbenchmark.py',
  'sid': 'autosklearn.benchmark.constraint.local.20211116T212125',
  'user_dir': '/tmp/tmpdyw0fb0t'}.
[DEBUG] [amlb.resources:21:21:25.561

['/tmp/tmp4027ws15-modep-output/autosklearn.benchmark.constraint.local.20211116T212125/logs/runbenchmark.20211116T212125.full.log']

## Flights

In [145]:
flights.train?

Signature:
flights.train(
    framework_names: List[str],
    train_ids: Union[str, List[str]],
    test_ids: Union[str, List[str]],
    target: str,
    max_runtime_seconds: int,
)
Docstring:
Start a job to train an AutoML framework flight.

:param list framework_names: The names of the frameworks to train
:param list train_ids: The ids of the dataset(s) to train on
:param list test_ids: The ids of the dataset(s) to test on
:param str target: The name of the target column
:param int max_runtime_seconds: The maximum amount of time in seconds to train per dataset(s)
:return: A :class:`modep_client.tasks.BaseTask` object
File:      ~/git/mlapi/modep_client/modep_client/automl.py
Type:      method


In [161]:
# maximum amount of time to train for
#max_runtime_seconds = 60 * 60
max_runtime_seconds = 60 * 10

flight_task = flights.train([], train_id, test_id, target, max_runtime_seconds)
flight_task

In [162]:
resp = flight_task.response
resp.pop('frameworks')
resp

{'id': 'fed6d277-f225-461d-8ba6-2c791c2bc163',
 'framework_names': ['AutoGluon',
  'AutoGluon_bestquality',
  'autosklearn',
  'autosklearn2',
  'AutoWEKA',
  'flaml',
  'GAMA',
  'H2OAutoML',
  'hyperoptsklearn',
  'mljarsupervised',
  'mljarsupervised_compete',
  'MLNet',
  'TPOT',
  'constantpredictor',
  'DecisionTree',
  'RandomForest',
  'TunedRandomForest'],
 'created': '2021-11-16 18:51:24.515214',
 'train_ids': ['4378de67-d256-4ed7-8653-82758ee1a343'],
 'test_ids': ['8042af36-76c0-4ba5-a7d5-af581d62e916'],
 'target': 'y',
 'max_runtime_seconds': 600,
 'status': 'RUNNING',
 'info': ''}

In [163]:
flight_id = flight_task.response['id']

In [197]:
flights.wait(flight_id)

flight status: SUCCESS
flight members:


,id,framework_name,version,created,status,problem_type,metric_name,metric_value,other_metrics,duration,training_duration,predict_duration,models_count,info
0,a15bec65-4519-4e58-9cc2-c1dd223293e6,AutoGluon,0.3.1,2021-11-16T18:51:24.580673,SUCCESS,regression,neg_rmse,-15.7150,"{'rmse': 15.715, 'r2': -0.536519, 'mae': 12.2274}",54.0,49.7,0.9,10,None
1,e819e05d-2439-4f7b-9dd1-c2cb874b6ced,AutoGluon_bestquality,0.3.1,2021-11-16T18:51:25.710678,SUCCESS,regression,neg_rmse,-15.7811,"{'rmse': 15.7811, 'r2': -0.549455, 'mae': 12.2887}",643.2,598.6,41.2,17,None
2,07309259-60ed-4b69-b927-01d5d1cd0925,autosklearn,0.13.0,2021-11-16T18:51:26.841652,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,ValueError: Found unknown categories ['none'] in column 0 during transform (no predictions)
3,febbbfb5-3172-4a87-b2ea-c2d73632859c,autosklearn2,0.13.0,2021-11-16T18:51:27.953792,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,ValueError: Found unknown categories ['none'] in column 0 during transform (no predictions)
4,21459838-27f2-4bf7-a25d-64529720ded0,AutoWEKA,2.6,2021-11-16T18:51:29.030912,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.3,0.0,0.0,0,ValueError: Regression is not supported. (no predictions)
5,e50789e5-5977-40ba-b276-3f0907bfe85a,flaml,0.6.2,2021-11-16T18:51:30.190110,SUCCESS,regression,neg_rmse,-15.8828,"{'rmse': 15.8828, 'r2': -0.569504, 'mae': 12.3307}",603.6,600.1,0.6,9,None
6,a14f2e44-afbb-4143-a47d-e17c55373651,GAMA,21.0.0,2021-11-16T18:51:31.292518,SUCCESS,regression,neg_rmse,-15.9752,"{'rmse': 15.9752, 'r2': -0.587824, 'mae': 12.4736}",542.4,540.0,0.1,50,None
7,f9ba222e-d0e2-4f08-a64c-b35841de0c3c,H2OAutoML,3.32.1.3,2021-11-16T18:51:32.401935,SUCCESS,regression,neg_rmse,-15.8286,"{'rmse': 15.8286, 'r2': -0.558811, 'mae': 12.31}",548.7,537.4,0.4,54,None
8,65c2df38-bdd0-4698-8169-f6002ecadac5,hyperoptsklearn,c66f8d,2021-11-16T18:51:33.517764,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,ValueError: Found unknown categories ['none'] in column 0 during transform (no predictions)
9,ad160d21-bca0-4fbc-a948-360633dd0fd3,mljarsupervised,0.10.4,2021-11-16T18:51:34.615822,SUCCESS,regression,neg_rmse,-15.7400,"{'rmse': 15.74, 'r2': -0.541395, 'mae': 12.2739}",242.4,232.8,0.9,7,None


In [198]:
flight = flights.get(flight_id)
trained_frameworks = pd.DataFrame(flight['frameworks'])

## Inspect created models

Some frameworks like AutoGluon and H2O have leaderboards, while others like TPOT and autosklearn have text descriptions.

In [199]:
for idx, row in trained_frameworks.iterrows():
    if row.fold_leaderboard:
        display(HTML(f'<h4>{row.framework_name} leaderboard</h4>'))
        # [0] is for the first and only test set fold
        lb = pd.DataFrame(row.fold_leaderboard[0])
        display(HTML(lb.to_html()))

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-11.432296,0.215704,14.152249,0.000770,0.499134,2,True,10
1,CatBoost,-11.435508,0.042569,4.043318,0.042569,4.043318,1,True,4
2,LightGBMXT,-11.546128,0.019783,0.915736,0.019783,0.915736,1,True,1
3,XGBoost,-11.549317,0.015005,0.686322,0.015005,0.686322,1,True,7
4,LightGBM,-11.561262,0.020909,0.589106,0.020909,0.589106,1,True,2
5,NeuralNetMXNet,-11.744794,0.451944,18.797242,0.451944,18.797242,1,True,8
6,LightGBMLarge,-11.824151,0.019855,1.412534,0.019855,1.412534,1,True,9
7,NeuralNetFastAI,-11.848478,0.151456,9.020691,0.151456,9.020691,1,True,6
8,RandomForestMSE,-12.099806,0.111828,5.487352,0.111828,5.487352,1,True,3
9,ExtraTreesMSE,-12.149248,0.111979,4.470185,0.111979,4.470185,1,True,5


,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-8.017009,9.455241,523.259115,0.000764,0.421307,3,True,17
1,LightGBMXT_BAG_L2,-8.061182,8.033216,478.293002,0.676614,97.318538,2,True,10
2,WeightedEnsemble_L2,-8.286719,6.260151,363.633348,0.000755,0.469921,2,True,9
3,CatBoost_BAG_L1,-8.293971,0.418015,30.230445,0.418015,30.230445,1,True,4
4,CatBoost_BAG_L2,-8.326987,7.782609,410.749910,0.426006,29.775447,2,True,13
5,LightGBM_BAG_L2,-8.372323,7.565960,390.624303,0.209358,9.649839,2,True,11
6,LightGBMXT_BAG_L1,-8.393383,0.202782,6.751935,0.202782,6.751935,1,True,1
7,XGBoost_BAG_L1,-8.426465,0.151915,6.369915,0.151915,6.369915,1,True,7
8,LightGBM_BAG_L1,-8.437195,0.197003,6.650739,0.197003,6.650739,1,True,2
9,LightGBMLarge_BAG_L2,-8.443674,7.584533,404.036949,0.227931,23.062486,2,True,16


,model_id,rmse,mean_residual_deviance,mse,mae,rmsle
0,XGBoost_grid__1_AutoML_20211116_192659_model_14,8.428627,71.041746,71.041746,5.345555,0.074529
1,StackedEnsemble_BestOfFamily_AutoML_20211116_192659,8.429322,71.053474,71.053474,5.363397,0.074537
2,StackedEnsemble_AllModels_AutoML_20211116_192659,8.430038,71.065546,71.065546,5.363069,0.074564
3,XGBoost_grid__1_AutoML_20211116_192659_model_27,8.444035,71.301719,71.301719,5.347553,0.074622
4,GBM_grid__1_AutoML_20211116_192659_model_4,8.457833,71.534933,71.534933,5.389645,0.074772
5,XGBoost_grid__1_AutoML_20211116_192659_model_4,8.472638,71.785600,71.785600,5.346919,0.074894
6,XGBoost_grid__1_AutoML_20211116_192659_model_8,8.475280,71.830373,71.830373,5.399831,0.075016
7,XGBoost_grid__1_AutoML_20211116_192659_model_17,8.514537,72.497345,72.497345,5.424752,0.075464
8,XGBoost_grid__1_AutoML_20211116_192659_model_6,8.521374,72.613817,72.613817,5.421333,0.075574
9,GBM_grid__1_AutoML_20211116_192659_model_9,8.526416,72.699765,72.699765,5.424918,0.075545


In [206]:
for idx, row in trained_frameworks.iterrows():
    if row.fold_model_txt:
        display(HTML(f'<h3>{row.framework_name} description</h3>'))
        # [0] is for the first and only test set fold
        display(row.fold_model_txt[0][:10])

## Inspect logs

In [202]:
failures = trained_frameworks.query(f"status != 'SUCCESS'")
failures

,id,framework_name,version,created,status,problem_type,metric_name,metric_value,other_metrics,duration,training_duration,predict_duration,models_count,fold_results,fold_leaderboard,fold_model_txt,info
2,07309259-60ed-4b69-b927-01d5d1cd0925,autosklearn,0.13.0,2021-11-16T18:51:26.841652,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,"[{'framework': 'autosklearn', 'version': '0.13...",None,None,ValueError: Found unknown categories ['none'] ...
3,febbbfb5-3172-4a87-b2ea-c2d73632859c,autosklearn2,0.13.0,2021-11-16T18:51:27.953792,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,"[{'framework': 'autosklearn2', 'version': '0.1...",None,None,ValueError: Found unknown categories ['none'] ...
4,21459838-27f2-4bf7-a25d-64529720ded0,AutoWEKA,2.6,2021-11-16T18:51:29.030912,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.3,0.0,0.0,0,"[{'framework': 'AutoWEKA', 'version': 2.6, 'fo...",None,None,ValueError: Regression is not supported. (no p...
8,65c2df38-bdd0-4698-8169-f6002ecadac5,hyperoptsklearn,c66f8d,2021-11-16T18:51:33.517764,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,"[{'framework': 'hyperoptsklearn', 'version': '...",None,None,ValueError: Found unknown categories ['none'] ...
12,0c841b91-6531-4b54-a71f-c5c09de380de,TPOT,0.11.7,2021-11-16T18:51:37.897888,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,"[{'framework': 'TPOT', 'version': '0.11.7', 'f...",None,None,ValueError: Found unknown categories ['none'] ...
14,f22bf16d-6e47-4f24-b320-770344836bd0,DecisionTree,0.24.2,2021-11-16T18:51:40.073521,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,"[{'framework': 'DecisionTree', 'version': '0.2...",None,None,ValueError: Found unknown categories ['none'] ...
15,ddd54cd4-a50d-497d-b496-9f28a916db9f,RandomForest,1.0.1,2021-11-16T18:51:41.161661,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.9,0.0,0.0,0,"[{'framework': 'RandomForest', 'version': '1.0...",None,None,ValueError: Found unknown categories ['none'] ...
16,8cd001fd-7597-4769-8eac-a554e9373d7f,TunedRandomForest,1.0.1,2021-11-16T18:51:42.275358,FAIL,regression,neg_rmse,NaN,"{'rmse': None, 'r2': None, 'mae': None}",0.8,0.0,0.0,0,"[{'framework': 'TunedRandomForest', 'version':...",None,None,ValueError: Found unknown categories ['none'] ...


In [207]:
for id in failures['id']:
    try:
        display(HTML(f'<h3>{id}</h3>'))
        frameworks.print_log(id)
    except Exception as e:
        logging.exception(e)
    break

[INFO] [amlb:19:04:56.716] Running benchmark `autosklearn` on `benchmark` framework in `local` mode.
[DEBUG] [amlb:19:04:56.717] Script args: Namespace(benchmark='benchmark', constraint='constraint', exit_on_error=False, extra=[], fold=None, framework='autosklearn', indir=None, keep_scores=True, logging='debug', mode='local', outdir='/tmp/tmp668a8nbg', parallel=1, profiling=False, session=None, setup='auto', task=None, userdir='/tmp/tmp668a8nbg').
[DEBUG] [amlb.utils.config:19:04:56.717] Loading config file `/bench/automlbenchmark/resources/config.yaml`.
[DEBUG] [amlb.utils.config:19:04:56.778] Loading config file `/tmp/tmp668a8nbg/config.yaml`.
[DEBUG] [amlb:19:04:56.780] Config args: { 'exit_on_error': False,
  'output_dir': '/tmp/tmp668a8nbg',
  'parallel_jobs': 1,
  'results': {'save': True},
  'run_mode': 'local',
  'script': 'runbenchmark.py',
  'sid': 'autosklearn.benchmark.constraint.local.20211116T190456',
  'user_dir': '/tmp/tmp668a8nbg'}.
[DEBUG] [amlb.resources:19:04:56.782

## Submit predictions

In [209]:
# select members for submission to kaggle
mems = trained_frameworks.query(f'status == "SUCCESS"')
len(mems)

9

In [211]:
# get test set predictions for each framework
preds = []
for _, row in mems.iterrows():
    preds_fwk = frameworks.predict(row['id'], test_id).result()
    print(row.framework_name, preds_fwk['status'])
    preds.append(preds_fwk['predictions'])

AutoGluon SUCCESS
AutoGluon_bestquality SUCCESS
flaml SUCCESS
GAMA SUCCESS
H2OAutoML SUCCESS
mljarsupervised SUCCESS
mljarsupervised_compete SUCCESS
MLNet SUCCESS
constantpredictor SUCCESS


In [212]:
pd.DataFrame(preds[0]).head()

,predictions,truth
0,100.833350,90.77
1,113.596850,91.44
2,98.400330,92.96
3,78.055725,89.40
4,110.660110,90.27


In [215]:
!head {ddir}/sample*.csv

ID,y
1,100.669318127821
2,100.669318127821
3,100.669318127821
4,100.669318127821
5,100.669318127821
8,100.669318127821
10,100.669318127821
11,100.669318127821
12,100.669318127821


In [216]:
classes = pd.DataFrame(preds[0]).columns[:-2]
classes

Index([], dtype='object')

In [ ]:
df_test = pd.read_csv(ddir + '/test.csv', usecols=['id'])

In [ ]:
df_train = pd.read_csv(ddir + '/train.csv', usecols=[target])

In [ ]:
label_to_int = {f'class_{i}': i for i in df_train[target].unique()}
label_to_int

In [ ]:
test_ids = df_test['id']
test_ids

In [ ]:
!mkdir submissions

In [ ]:
import uuid
submission_id = str(uuid.uuid4())[:5]
submission_id

In [ ]:
# for i, (_, row) in enumerate(mems.iterrows()):
#     print('\n' + row.framework_name)
    
#     # The last two columns are predicted class and ground truth class, thus the `.iloc[:, :-2]`.
#     probs = pd.DataFrame(preds[i]).iloc[:, :-2]
#     assert np.allclose(probs.sum(1).values, 1)
    
#     if len(classes) == 2:
#         # binary
#         submission = pd.DataFrame(probs.values[:,1], columns=['target'], index=test_ids)
#     else:
#         # multi-class
#         submission = pd.DataFrame(probs.values, columns=classes, index=test_ids)
    
#     fname = f'submissions/{submission_id}-{row.framework_name}.csv'
#     submission.to_csv(fname)
    
#     !kaggle competitions submit $dataset -f {fname} -m {fname}
    
#     time.sleep(60)

In [ ]:
# !kaggle competitions submissions -q $dataset --csv > submissions/table.csv

In [ ]:
# pd.read_csv(f"submissions/table.csv").head(len(mems))#.sort_values(by='publicScore')

## Cleanup

In [14]:
if 1:
    ## delete all datasets
    for idx in datasets.list().index:
        print(datasets.delete(idx))
    
    ## delete all framework runs
    for idx in frameworks.list().index:
        print(frameworks.delete(idx))
        
    ## delete all flights
    for idx in flights.list().index:
        print(flights.delete(idx))    